In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import TensorBuilder
from utils.acdc import cg_combine
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion

ModuleNotFoundError: No module named 'aml_storage'

In [ ]:
frames = ase.io.read("data/hamiltonian/water-hamiltonian/water_coords_1000.xyz",":")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [ ]:
jorbs = json.load(open('data/hamiltonian/water-hamiltonian/orbs_def2_water.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [ ]:
hams = np.load("data/hamiltonian/water-hamiltonian/water_fock.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

In [ ]:
blocks = dense_to_blocks(hams, frames, orbs)

In [ ]:
blocks.keys

Labels([( 0, 8, 1, 0, 8, 1, 0), ( 0, 8, 1, 0, 8, 2, 0),
        ( 0, 8, 1, 0, 8, 3, 0), ( 0, 8, 1, 0, 8, 2, 1),
        ( 0, 8, 1, 0, 8, 3, 1), ( 0, 8, 1, 0, 8, 3, 2),
        ( 0, 8, 2, 0, 8, 2, 0), ( 0, 8, 2, 0, 8, 3, 0),
        ( 0, 8, 2, 0, 8, 2, 1), ( 0, 8, 2, 0, 8, 3, 1),
        ( 0, 8, 2, 0, 8, 3, 2), ( 0, 8, 3, 0, 8, 3, 0),
        ( 0, 8, 3, 0, 8, 3, 1), ( 0, 8, 3, 0, 8, 3, 2),
        ( 0, 8, 2, 1, 8, 3, 0), ( 0, 8, 2, 1, 8, 2, 1),
        ( 0, 8, 2, 1, 8, 3, 1), ( 0, 8, 2, 1, 8, 3, 2),
        ( 0, 8, 3, 1, 8, 3, 1), ( 0, 8, 3, 1, 8, 3, 2),
        ( 0, 8, 3, 2, 8, 3, 2), ( 2, 1, 1, 0, 8, 1, 0),
        ( 2, 1, 1, 0, 8, 2, 0), ( 2, 1, 1, 0, 8, 3, 0),
        ( 2, 1, 1, 0, 8, 2, 1), ( 2, 1, 1, 0, 8, 3, 1),
        ( 2, 1, 1, 0, 8, 3, 2), ( 2, 1, 2, 0, 8, 1, 0),
        ( 2, 1, 2, 0, 8, 2, 0), ( 2, 1, 2, 0, 8, 3, 0),
        ( 2, 1, 2, 0, 8, 2, 1), ( 2, 1, 2, 0, 8, 3, 1),
        ( 2, 1, 2, 0, 8, 3, 2), ( 2, 1, 2, 1, 8, 1, 0),
        ( 2, 1, 2, 1, 8, 2, 0), ( 2, 1, 2, 1, 8,

In [ ]:
cg = ClebschGordanReal(4)

In [ ]:
coupled = couple_blocks(blocks, cg)

In [ ]:
decoupled = decouple_blocks(coupled, cg)

In [ ]:
dense = blocks_to_dense(decoupled, frames, orbs)

In [ ]:
np.linalg.norm(dense[10] - hams[10])

4.944143975439669e-15

In [ ]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

In [ ]:
spex = RascalSphericalExpansion(rascal_hypers)
pairs = RascalPairExpansion(rascal_hypers)

In [ ]:
rhoi = spex.compute(frames)
gij = pairs.compute(frames)

In [ ]:
rhoi.keys

Labels([(0, 1, 1), (0, 1, 8), (0, 8, 1), (0, 8, 8), (1, 1, 1), (1, 1, 8),
        (1, 8, 1), (1, 8, 8), (2, 1, 1), (2, 1, 8), (2, 8, 1), (2, 8, 8)],
       dtype=[('spherical_harmonics_l', '<i4'), ('species_center', '<i4'), ('species_neighbor', '<i4')])

In [ ]:
gij.block(2).samples

Labels([(  0, 0, 0), (  0, 0, 1), (  0, 0, 2), ..., (999, 2, 0),
        (999, 2, 1), (999, 2, 2)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [ ]:
rhoi.block(spherical_harmonics_l=1, species_center=8, species_neighbor=8).properties#values.shape

Labels([(0,), (1,), (2,)], dtype=[('n', '<i4')])

In [ ]:
for idx, block in coupled:
    btype, ai, ni, li, aj, nj, lj, L = tuple(idx)
    if np.abs(btype) == 0:
        if (np.linalg.norm(block.values) < 1e-10):
            print(btype, ni, nj, li, lj, (-1)**(li+lj+L), np.linalg.norm(block.values))

In [ ]:
ki, kj = 0, 0
ham_builder = TensorBuilder(["a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j"], [["m1"], ["m2"]], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        for ni, li, mi in jorbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i>j:
                    continue
                aj = zdic[fj]
                for nj, lj, mj in jorbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(keys=block_idx,properties=np.asarray([[0]],dtype=np.int32), 
                            components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )] )                                                    
                    else:
                        block = ham_builder.blocks[block_idx]
                    
                    block.add_samples(labels=[(A,i,j)], 
                                      data=np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1]).reshape((1,2*li+1,2*lj+1,1)))

                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [ ]:
ki, kj = 0, 0
ham_builder = TensorBuilder(["block_type", "a1", "n1", "l1", "a2", "n2", "l2"], ["structure", "atom_i", "atom_j", "ki_base", "kj_base"], [["m1"], ["m2"]], ["hamiltonian"])
zdic = {"O": 8, "H": 1}
for A in range(len(frames)):
    frame = frames[A]
    ham = hams[A]
    ki = 0
    for i, fi in enumerate(frame.symbols):
        ai = zdic[fi]
        ki_base = ki # pointer at where the i-atom block starts
        for ni, li, mi in jorbs[fi]:
            kj = 0
            if mi != -li:
                continue
            for j, fj in enumerate(frame.symbols):
                if i<j: # operate only on the lower-triangular block
                    continue                    
                aj = zdic[fj]
                if i==j:
                    block_type = 0  # diagonal
                elif ai==aj:
                    block_type = 1  # same-species
                else:
                    block_type = 2  # different species
                kj_base = kj # pointer at where the j-atom block starts
                for nj, lj, mj in jorbs[fj]:                
                    if mj != -lj:
                        continue
                    block_idx = (block_type, ai, ni, li, aj, nj, lj)
                    if block_idx not in ham_builder.blocks:
                        block = ham_builder.add_block(keys=block_idx, properties=np.asarray([[0]], dtype=np.int32), 
                             components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )])                                                    
                        if block_type == 1:
                            block_asym = ham_builder.add_block(keys=(-1,)+block_idx[1:], properties=np.asarray([[0]], dtype=np.int32), 
                            components=[np.asarray(list(product(range(-li,li+1)) ), dtype=np.int32 ),
                                        np.asarray(list(product(range(-lj,lj+1)) ), dtype=np.int32 )])                                                    
                    else:                        
                        block = ham_builder.blocks[block_idx]
                        if block_type == 1:
                            block_asym = ham_builder.blocks[(-1,)+block_idx[1:]]
                    
                    block_data_ij = np.asarray(ham[ki:ki+2*li+1, kj:kj+2*lj+1])

                    if block_type == 1:
                        kj_offset = kj-kj_base
                        ki_offset = ki-ki_base
                        block_data_ji = np.asarray(ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1])                        
                        block.add_samples(labels=[(A,i,j, ki_base, kj_base)], data=(block_data_ij+block_data_ji).reshape((1,2*li+1,2*lj+1,1))/np.sqrt(2) )
                        block_asym.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=(block_data_ij-block_data_ji).reshape((1,2*li+1,2*lj+1,1))/np.sqrt(2) )
                    else:
                        block.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=block_data_ij.reshape((1,2*li+1,2*lj+1,1)))                    
                    kj += 2*lj+1
            ki += 2*li+1
ham_et = ham_builder.build()

In [ ]:
np.linalg.norm(dense[10] - hams[10])

4.944143975439669e-15

In [ ]:
dense = []
zdic = {8:"O", 1:"H"}
for f in frames:
    norbs = 0
    for ai in f.symbols:        
        norbs += len(jorbs[ai])
    ham = np.zeros((norbs, norbs), dtype=np.float64)
    dense.append(ham)

for idx, block in ham_et:
    cur_A = -1
    block_type, ai, ni, li, aj, nj, lj = tuple(idx)
    fi = zdic[ai]
    fj = zdic[aj]
    ki_offset = 0
    for no, lo, mo in jorbs[fi]:        
        if no == ni and lo == li:
            break
        ki_offset += 1
    kj_offset = 0
    for no, lo, mo in jorbs[fj]:        
        if no == nj and lo == lj:
            break
        kj_offset += 1
    for (A,i,j,ki_base,kj_base), block_data in zip(block.samples, block.values):
        if A != cur_A:
            ham = dense[A]
            cur_A = A
        if block_type == 0:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
        elif block_type == 2:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1)
            ham[kj_base+kj_offset:kj_base+kj_offset+2*lj+1, ki_base+ki_offset:ki_base+ki_offset+2*li+1] = block_data[:,:,0].reshape(2*li+1,2*lj+1).T
        elif block_type == 1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1)  / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        elif block_type == -1:
            ham[ki_base+ki_offset:ki_base+ki_offset+2*li+1, kj_base+kj_offset:kj_base+kj_offset+2*lj+1] += np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
            ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1] -= np.asarray(block_data[:,:,0].reshape(2*li+1,2*lj+1) / np.sqrt(2), dtype=np.float64)
        



In [ ]:
np.linalg.norm(dense[10] - hams[10])

2.8031757299712627e-16